In [1]:
pushd  ../../lux

/Users/dorislee/Desktop/Research/lux/lux


['~/Desktop/Research/lux/lux-benchmark/accuracy']

In [2]:
import lux
import pandas as pd

In [3]:
popd

/Users/dorislee/Desktop/Research/lux/lux-benchmark/accuracy
popd -> ~/Desktop/Research/lux/lux-benchmark/accuracy


In [4]:
df = pd.read_csv("../../lux-datasets/data/airbnb_nyc.csv")

In [5]:
all_recs1 = df.recommendation
l1 = all_recs1["Correlation"]
l1_scores= [vis.score for vis in l1] 

/Users/dorislee/Desktop/Research/lux/lux/lux/executor/PandasExecutor.py:396: UserWarning:
Lux detects that the attribute 'last_review' may be temporal.
To display visualizations for these attributes accurately, please convert temporal attributes to Pandas Datetime objects using the pd.to_datetime function and provide a 'format' parameter to specify the datetime format of the attribute.
For example, you can convert a year-only attribute (e.g., 1998, 1971, 1982) to Datetime type by specifying the `format` as '%Y'.

Here is a starter template that you can use for converting the temporal fields:
	df['last_review'] = pd.to_datetime(df['last_review'], format='<replace-with-datetime-format>')

See more at: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html


In [31]:
def get_hash(vis:lux.vis.Vis)-> str:
    # Get the visualization hash for a Vis object
    # Vis(...) --> 'Vis  (x: reviews_per_month, y: number_of_reviews)'
    vis_hash = vis.__repr__()
    return vis_hash.split("mark")[0][1:-1]

In [7]:
def convert_vlist_to_hashmap(vlist):
    # Convert VisList to dictionary representation
    # {'Vis  (x: ..., y: ...)': 0.55,
    #  'Vis  (x: ..., y: ...)': 0.22,
    #   ...}
    vdict = {}
    for vis in vlist:
        vdict[get_hash(vis)]=vis.score
    return vdict

In [8]:
map1 = convert_vlist_to_hashmap(l1)

In [12]:
lux.config.sampling_start=1000
lux.config.sampling_cap=1000

In [13]:
df = pd.read_csv("../../lux-datasets/data/airbnb_nyc.csv")

In [14]:
all_recs2 = df.recommendation
l2 = all_recs2["Correlation"]
l2_scores= [vis.score for vis in l2] 

/Users/dorislee/Desktop/Research/lux/lux/lux/executor/PandasExecutor.py:396: UserWarning:
Lux detects that the attribute 'last_review' may be temporal.
To display visualizations for these attributes accurately, please convert temporal attributes to Pandas Datetime objects using the pd.to_datetime function and provide a 'format' parameter to specify the datetime format of the attribute.
For example, you can convert a year-only attribute (e.g., 1998, 1971, 1982) to Datetime type by specifying the `format` as '%Y'.

Here is a starter template that you can use for converting the temporal fields:
	df['last_review'] = pd.to_datetime(df['last_review'], format='<replace-with-datetime-format>')

See more at: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html


In [15]:
map2 = convert_vlist_to_hashmap(l2)

In [16]:
# Combine two dictionaries map1,map2 into a single global_map
global_map = set(map1.keys())
global_map.update(set(map2.keys()))
global_map = list(global_map)

In [17]:
def get_aligned_dict(vdict,global_map):
    # Align each vdict based on global map
    aligned_dict = {}
    for vis in global_map: 
        if vis in vdict:
            aligned_dict[vis] = vdict[vis]
        else:
            aligned_dict[vis] = 0
    return aligned_dict

In [18]:
aligned_score1 = list(get_aligned_dict(map1,global_map).values())
aligned_score2 = list(get_aligned_dict(map2,global_map).values())

In [19]:
print(aligned_score1)
print(aligned_score2)

[0.1549660456513047, 0.22353268112451927, 0.18529648835900278, 0, 0.07353391911130251, 0, 0.14948361961918694, 0.08001873902011758, 0.5523251276922889, 0.13909879991644414, 0.07646134642069562, 0.12384225515403763, 0.11485253583270018, 0.08100472040464925, 0, 0.17183135618414175, 0.08438800885276339, 0.11436329320441341]
[0.09451102052982349, 0.20082318605690364, 0.19016713526380277, 0.17815544514759957, 0, 0.09136079805796612, 0.2249326664201778, 0.08638548507827103, 0.5418055143452527, 0.1847275230884106, 0.1454363645849904, 0.2053266446950923, 0.09656250622578202, 0, 0.10682804392248306, 0.1452131486772297, 0, 0.18404539359844904]


In [20]:
from scipy.stats import stats

In [21]:
rank1 = stats.rankdata(aligned_score1)
rank2 =stats.rankdata(aligned_score2)

In [22]:
# from lux.utils.utils import get_filter_specs
# def equals(self,other):
#     x_equals = clause_equals(self.get_attr_by_channel("x")[0],other.get_attr_by_channel("x")[0])
#     y_equals = clause_equals(self.get_attr_by_channel("y")[0],other.get_attr_by_channel("y")[0])
#     if (len(self.get_attr_by_channel("color"))>0):
#         color_equals = clause_equals(self.get_attr_by_channel("color")[0],other.get_attr_by_channel("color")[0])
#     else:
#         color_equals = True
#     filter_equals = get_filter_specs(self.intent) == get_filter_specs(other.intent)
#     mark_equals = self.mark == other.mark
#     #print (x_equals,y_equals,color_equals,filter_equals)
#     return x_equals and y_equals and color_equals and filter_equals and mark_equals

# def clause_equals(self, other):
#     props = ['aggregation', 'attribute', 'bin_size', 'channel', 'description', 'exclude', 'filter_op', 'sort', 'value', 'weight']
#     flag = True
#     for prop in props:
#         eq = getattr(self,prop) == getattr(other,prop)
#         flag=flag and eq
#     return flag

# equals(l1[0],l2[0])

In [25]:
import numpy as np
def dcg(r, k, method=0,debug=False):
    # alternative formulation of DCG places stronger emphasis on retrieving relevant documents
    r = np.asfarray(r)[:k]
    val = 0
    for i in range(1,len(r)+1):
        val+= (r[i-1]) / np.log2(i+1)
        if debug:
            print ("i=",i,":",(r[i-1]) ,"/", np.log2(i+1))
            print ((r[i-1]) / np.log2(i+1))
    return val

def ndcg(ground_truth_r,r, k,debug=False):
    return dcg(r, k,debug=debug) / dcg(ground_truth_r,k,debug=debug)

In [28]:
ndcg(aligned_score1,aligned_score2,3) 

0.8138473582262048

In [30]:
# if we only care about ranks`
ndcg(rank1,rank2,3) 

0.6864291267112796

In [66]:
ground_truth_ratings = [1,2,3,4,5,6]
example_ratings = [3,2,3,0,1,2]
ideal_ordering = [3,3,3,2,2,2,1,0]
assert np.isclose(dcg(example_ratings,6),6.861,1e-2) #check DCG calculation (Based on Wikipedia example)
assert np.isclose(ndcg(ideal_ordering,[3,2,3,0,1,2],6),0.785,1e-2) #check NDCG calculation (Based on Wikipedia example)
assert np.isclose(ndcg([3,1,2],[3, 1, 2],3),1) #sanity check